In [ ]:
# # use erddapy to get some data
# from erddapy import ERDDAP
# import numpy as np
#
# e = ERDDAP(
#     server="https://erddap.marine.usf.edu/erddap",
#     protocol="griddap",
# )
#
# e.dataset_id = "jplMURSST41anom1day"
# print(f"variables in this dataset:\n\n{e.variables}")
#
# # set bounds to reduce download size
# e.constraints = {
#     'time>=': '2003-06-07T09:00:00Z',
#     'time<=': '2024-06-07T09:00:00Z',
#     'time_step': 1,
#     'latitude>=': np.float64(lat - 1),
#     'latitude<=': np.float64(lat + 1),
#     'latitude_step': 1,
#     'longitude>=': np.float64(lon - 1),
#     'longitude<=': np.float64(lon + 1),
#     'longitude_step': 1
# }
#
# Download as a NetCDF to disk
# nc_filename = e.download_file(".nc")
# (under the hood this uses the GRiDDAP `.nc` URL)
#
# ds = e.to_xarray()

In [4]:
# setup
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install dask pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# sample all taxa occurrence points for ERDDAP data values:
# data has already been downloaded as .nc
# example:
# https://erddap.marine.usf.edu/erddap/griddap/cmems_salinity.graph?so%5B(2023-06-16)%5D%5B(0.494025)%5D%5B(-80.0):(90.0)%5D%5B(-180.0):(179.9167)%5D&.draw=surface&.vars=longitude%7Clatitude%7Cso&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff

import pickle

import dask.dataframe as dd
with open('/content/drive/MyDrive/occurrences.pkl', 'rb') as f:
    occurrences = pickle.load(f)  # pandas.df

import xarray as xr
ds = xr.open_dataset(
    "/content/drive/MyDrive/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc",
    chunks={"time": 1, "latitude": 500, "longitude": 500}
)


# build a Dask DataFrame of salinities
sal = ds["so"].sel(
    latitude=occurrences["decimalLatitude"],
    longitude=occurrences["decimalLongitude"],
    method="nearest"
)
# this is a dask Array; turn it into a dask Series
occurrences["salinity"] = sal.to_dask_dataframe(name="salinity")["salinity"]

# trigger computation in manageable chunks:
result = occurrences.compute()
print(result.head())

# for occurrence in occurrences.iterrows():
#     # adjust names if dims are named 'latitude'/'longitude' or 'lat'/'lon'
#     print(occurrence["decimalLatitude"])
#     val = ds["so"].sel(
#         latitude=occurrence["decimalLatitude"],
#         longitude=occurrence["decimalLongitude"],
#         method="nearest"
#     ).values

#     print(f"At ({occurrence["decimalLatitude"]}, {occurrence["decimalLongitude"]}), so = {val}")

# lat = 60
# lon = -60

# # Open file with xarray
# ds = xr.open_dataset("data/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc")


# # adjust names if dims are named 'latitude'/'longitude' or 'lat'/'lon'
# val = ds["so"].sel(
#     latitude=lat,
#     longitude=lon,
#     method="nearest"
# ).values

# print(f"At ({lat}, {lon}), so = {val}")

ValueError: cannot use a dict-like object for selection on a dimension that does not have a MultiIndex